In [1]:
# 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import sys


clickmodule_dir = "/home/jovyan/eb/Documents/inbox science/clicks_nbdev/"
sys.path.append(clickmodule_dir)

#os.chdir(clickmodule_dir)

from clicks_nbdev.offline_augmentation import asnLookup

import clicks_nbdev.el_augmentation as ela

In [4]:
# Create some instances we'll need
p = ela.requestdict_parser()
asndb = asnLookup()

In [5]:
asndb.get_asn_details('172.56.20.104')

(21928, '172.56.20.0/23', 'T-MOBILE-AS21928, US')

In [6]:
print(os.getcwd())

/home/jovyan/eb/Documents/inbox science/maawg/m3_sf_2020_training


In [7]:
#parquet_path = "/home/jovyan/eb/Documents/inbox science/clicks_nbdev/temp/"

parquet_path = "/home/jovyan/eb/Documents/inbox science/clicks_nbdev/temp/step2"


In [8]:
parquet_files = []
for i in os.listdir(parquet_path):
    if i.startswith("prod_") and i.endswith("parquet"):
        parquet_files.append(i)

In [9]:
def extract_farm(filename):
    return i.replace("prod_aem_", "").split("-")[0]

In [10]:
parquet_files[0:6]

['prod_aem_elabs10-web10_access_log-20200112_step2.parquet',
 'prod_aem_elabs10-web1_access_log-20200119_step2.parquet',
 'prod_aem_elabs10-web1_access_log-20200126_step2.parquet',
 'prod_aem_elabs10-web2_access_log-20200105_step2.parquet',
 'prod_aem_elabs10-web2_access_log-20200112_step2.parquet',
 'prod_aem_elabs10-web3_access_log-20200126_step2.parquet']

In [19]:
files_to_load = 25

df_loaded = None
#pd.read_parquet(parquet_path + "/" + parquet_files[0])
for n, i in enumerate(parquet_files[0:files_to_load]):
    
    this_farm = extract_farm(i)
    
    _df = pd.read_parquet(parquet_path + "/" + i)
    _df = _df.assign(farm=this_farm)
    print(n, "shape:", _df.shape, i)
    if isinstance(df_loaded, pd.core.frame.DataFrame):
        df_loaded = pd.concat([df_loaded, _df])
    else:
        df_loaded = _df

del _df


# Some cleanup... 
#df_loaded = df.drop('ip_slash24', axis='columns')

df_loaded.drop('request_header_host', axis='columns', inplace=True)

# concat farm + mid for uniqueness
df['farm_mid'] = df['farm'] + "_" + df['mid'].astype(str)

# drop any images that managed to sneak by
_flt = df_loaded.query('el_request_type == "image"')
df_loaded.drop(_flt.index, inplace=True)

df_loaded.rename({"time_received_utc_datetimeobj":"request_time"}, axis='columns', inplace=True)

print("final shape:", df_loaded.shape)

tf: elabs10
0 shape: (56820, 26) prod_aem_elabs10-web10_access_log-20200112_step2.parquet
tf: elabs10
1 shape: (67296, 26) prod_aem_elabs10-web1_access_log-20200119_step2.parquet
tf: elabs10
2 shape: (90432, 26) prod_aem_elabs10-web1_access_log-20200126_step2.parquet
tf: elabs10
3 shape: (45297, 26) prod_aem_elabs10-web2_access_log-20200105_step2.parquet
final shape: (259845, 25)


In [20]:
df = df_loaded.copy()

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 259845 entries, 16 to 332147
Data columns (total 25 columns):
request_time                                          259845 non-null datetime64[ns, UTC]
remote_ip                                             259845 non-null object
conn_status                                           259845 non-null object
request_method                                        259845 non-null object
request_url_path                                      259845 non-null object
request_url_query_simple_dict                         259845 non-null object
status                                                259845 non-null object
response_bytes_clf                                    259845 non-null object
request_header_referer                                259845 non-null object
request_header_user_agent                             259845 non-null object
request_header_user_agent__browser__family            259845 non-null object
request_header_user_agent__b

In [30]:
df.head()

,request_time,remote_ip,conn_status,request_method,request_url_path,request_url_query_simple_dict,status,response_bytes_clf,request_header_referer,request_header_user_agent,...,el_request_type,sid,mid,mlid,ruid,asn,netblock,as_org,farm,farm_mid
index,,,,,,,,,,,,,,,,,,,,,
16,2020-01-05 03:42:16+00:00,209.73.183.25,+,GET,/images/mlopen_post.html,"{'rtr': 'on', 'siteid': '2010000698', 'mid': '...",200,42,-,YahooMailProxy; https://help.yahoo.com/kb/yaho...,...,open,2010000698,5985220,365247,8f540717cf,26101,209.73.180.0/22,"YAHOO-3, US",elabs10,elabs105985220
20,2020-01-05 03:42:17+00:00,66.249.84.56,+,GET,/images/mlopen_post.html,"{'rtr': 'on', 'siteid': '2010003189', 'mid': '...",200,42,-,Mozilla/5.0 (Windows NT 5.1; rv:11.0) Gecko Fi...,...,open,2010003189,5935138,323385,aafd4b9735,15169,66.249.84.0/23,"GOOGLE, US",elabs10,elabs105935138
31,2020-01-05 03:42:36+00:00,172.56.20.104,+,GET,/c.html,"{'ufl': '1', 'rtr': 'on', 's': 'x8pbpz,344ep,8...",302,26,-,Mozilla/5.0 (Linux; Android 8.1.0; LM-Q710(FGN...,...,click_html,2010002327,5231185,407154,1ccb663e58,21928,172.56.20.0/23,"T-MOBILE-AS21928, US",elabs10,elabs105231185
34,2020-01-05 03:42:41+00:00,172.219.32.58,+,GET,/images/mlopen_post.html,"{'rtr': 'on', 'siteid': '2010000698', 'mid': '...",200,42,-,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6...,...,open,2010000698,5967275,459998,90771b26e1,852,172.219.0.0/16,"ASN852, CA",elabs10,elabs105967275
40,2020-01-05 03:42:54+00:00,66.249.81.28,+,GET,/images/mlopen_post.html,"{'rtr': 'on', 'siteid': '2010002327', 'mid': '...",200,42,-,Mozilla/5.0 (Windows NT 5.1; rv:11.0) Gecko Fi...,...,open,2010002327,5286515,215864,3a08801d81,15169,66.249.80.0/22,"GOOGLE, US",elabs10,elabs105286515


In [29]:
df['farm_mid'] = df['farm'] + "_" + df['mid'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 259845 entries, 16 to 332147
Data columns (total 26 columns):
request_time                                          259845 non-null datetime64[ns, UTC]
remote_ip                                             259845 non-null object
conn_status                                           259845 non-null object
request_method                                        259845 non-null object
request_url_path                                      259845 non-null object
request_url_query_simple_dict                         259845 non-null object
status                                                259845 non-null object
response_bytes_clf                                    259845 non-null object
request_header_referer                                259845 non-null object
request_header_user_agent                             259845 non-null object
request_header_user_agent__browser__family            259845 non-null object
request_header_user_agent__b

In [268]:
#dtest = df.head(1000).copy()

In [20]:
for i in ['sid', 'mid', 'mlid', 'ruid']:
    if i not in df.columns:
        # Parse out EL details
        p.estimate(df.shape[0])
        %time df[dcols] = p.parse_df(df[['el_request_type', 'request_url_query_simple_dict']])

In [22]:
# Get ASN / netblock info from the IP
asn_cols = ['asn', 'netblock', 'as_org']
for i in asn_cols:
    if i not in df.columns:
        df[asn_cols] = pd.DataFrame(df['remote_ip'].apply(asndb.get_asn_details).tolist(), index=df.index)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537572 entries, 16 to 269777
Data columns (total 24 columns):
request_time                                          1537572 non-null datetime64[ns, UTC]
remote_ip                                             1537572 non-null object
conn_status                                           1537572 non-null object
request_header_host                                   1537572 non-null object
request_method                                        1537572 non-null object
request_url_path                                      1537572 non-null object
request_url_query_simple_dict                         1537572 non-null object
status                                                1537572 non-null object
response_bytes_clf                                    1537572 non-null object
request_header_referer                                1537572 non-null object
request_header_user_agent                             1537572 non-null object
request_header_u

In [311]:
asndb = asnLookup()
asndb.get_asn_details('172.56.20.104')

(21928, '172.56.20.0/23', 'T-MOBILE-AS21928, US')

In [43]:
df.request_header_user_agent.value_counts().head(5)

Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko)                                           29041
Mozilla/5.0 (Windows NT 5.1; rv:11.0) Gecko Firefox/11.0 (via ggpht.com GoogleImageProxy)                        21183
Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148    17542
YahooMailProxy; https://help.yahoo.com/kb/yahoo-mail-proxy-SLN28749.html                                          6855
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15) AppleWebKit/605.1.15 (KHTML, like Gecko)                            3602
Name: request_header_user_agent, dtype: int64

In [42]:
df.request_header_user_agent__browser__family.value_counts().head(7)

Other                         40944
Mobile Safari UI/WKWebView    24779
GmailImageProxy               21183
Apple Mail                    10281
Chrome                         9005
Outlook                        8131
IE                             7011
Name: request_header_user_agent__browser__family, dtype: int64

In [21]:
df.el_request_type.value_counts()

open               152611
click_html          21677
unknown             10222
click_text           5056
unsubscribe          1192
unsub_conf_page       761
image                   6
Name: el_request_type, dtype: int64

In [29]:
top_ips = df.remote_ip.value_counts().head(500)
# value_counts to df:
top_ips = top_ips.rename_axis('remote_ip').reset_index(name='count')

In [69]:
def df_counts(df, column, head=None):
    # returns a df of value_counts of the column.
    col = str(column)
    if head:
        top_values = df[column].value_counts().head(head)
    else:
        top_values = df[column].value_counts()
        
    colname = col + "_count"
    top_values = top_values.rename_axis(column).reset_index(name=colname)
    top_values[column] = top_values[column].astype('int32')
    return top_values

In [328]:
%time df[['asn', 'netblock', 'as_org']] = pd.DataFrame(df['remote_ip'].apply(asndb.get_asn_details).tolist(), index=df.index)
df['asn'] = df['asn'].astype('int32')

CPU times: user 943 ms, sys: 0 ns, total: 943 ms
Wall time: 942 ms


In [329]:
df.as_org.value_counts().head(15)

GOOGLE, US                         32875
MAILRU-AS Mail.Ru, RU              29371
COMCAST-7922, US                   11407
AMAZON-02, US                       8663
MICROSOFT-CORP-MSN-AS-BLOCK, US     3867
ATT-INTERNET4, US                   3614
BACOM, CA                           3150
CELLCO, US                          3049
ATT-MOBILITY-LLC-AS20057, US        2695
BELGACOM-SKYNET-AS, BE              2562
ASN852, CA                          2542
INKTOMI-LAWSON, US                  2484
ROGERS-COMMUNICATIONS, CA           2239
UUNET, US                           2033
SHAW, CA                            1989
Name: as_org, dtype: int64

In [22]:
df.head()

,time_received_utc_datetimeobj,remote_ip,conn_status,request_header_host,request_method,request_url_path,request_url_query_simple_dict,status,response_bytes_clf,request_header_referer,...,request_header_user_agent__browser__family,request_header_user_agent__browser__version_string,request_header_user_agent__os__family,request_header_user_agent__os__version_string,request_header_user_agent__is_mobile,ip_slash24,el_request_type,asn,netblock,as_org
index,,,,,,,,,,,,,,,,,,,,,
16,2020-01-05 03:42:16+00:00,209.73.183.25,+,www.elabs10.com,GET,/images/mlopen_post.html,"{'rtr': 'on', 'siteid': '2010000698', 'mid': '...",200,42,-,...,Other,,Other,,False,209.73.183.0,open,26101,209.73.180.0/22,"YAHOO-3, US"
20,2020-01-05 03:42:17+00:00,66.249.84.56,+,click.crewardsmailing.com,GET,/images/mlopen_post.html,"{'rtr': 'on', 'siteid': '2010003189', 'mid': '...",200,42,-,...,GmailImageProxy,,Windows,XP,False,66.249.84.0,open,15169,66.249.84.0/23,"GOOGLE, US"
31,2020-01-05 03:42:36+00:00,172.56.20.104,+,www.elabs10.com,GET,/c.html,"{'ufl': '1', 'rtr': 'on', 's': 'x8pbpz,344ep,8...",302,26,-,...,Chrome Mobile,79.0.3945,Android,8.1.0,True,172.56.20.0,click_html,21928,172.56.20.0/23,"T-MOBILE-AS21928, US"
34,2020-01-05 03:42:41+00:00,172.219.32.58,+,www.elabs10.com,GET,/images/mlopen_post.html,"{'rtr': 'on', 'siteid': '2010000698', 'mid': '...",200,42,-,...,Apple Mail,605.1.15,Mac OS X,10.14.6,False,172.219.32.0,open,852,172.219.0.0/16,"ASN852, CA"
40,2020-01-05 03:42:54+00:00,66.249.81.28,+,click.rewardsdealmail.com,GET,/images/mlopen_post.html,"{'rtr': 'on', 'siteid': '2010002327', 'mid': '...",200,42,-,...,GmailImageProxy,,Windows,XP,False,66.249.81.0,open,15169,66.249.80.0/22,"GOOGLE, US"


In [23]:
df.netblock.value_counts().head()

188.93.56.0/21     62721
209.222.82.0/24    49788
74.125.0.0/16      45611
64.233.172.0/23    40571
66.102.8.0/23      33378
Name: netblock, dtype: int64

In [24]:
n_by_netblock = pd.DataFrame(df.groupby(["netblock", "as_org", "asn"])["netblock"].count())

In [25]:
n_by_netblock = n_by_netblock.rename(columns={"netblock":"netblock_count"})

# Groupby convers the grouped columns to the index, so we reset them to regular columns.
n_by_netblock = n_by_netblock.sort_values('netblock_count', axis='index', ascending=False).reset_index()

In [ ]:
def value_counts_multi(df, group_column, return_columns):
    # Returns a new df, grouped by specified columns, returning return_columns, sorted by count descending.
    # return_columns should be a list.
    
    df1 = pd.DataFrame(df.groupby(return_columns)[group_column].count())
    df1 = df1.rename(columns={group_column:"netblock_count"})
    
    
    

In [37]:
cdf = df.query('(el_request_type == "click_html" or el_request_type == "click_text") and mid > 0').copy()

print(cdf.shape)

test = 0

if test:
    cdf = cdf.head(5000).copy()

(200235, 24)


In [25]:
cdf.info()

NameError: name 'cdf' is not defined

In [39]:
# Let's just get the top 10 IPs that are clicking.
cdf['remote_ip'].value_counts().head(10)

169.61.122.229    5139
65.154.226.220    4921
65.154.226.100    3562
65.155.30.101     2545
65.154.226.109    1601
209.222.82.140    1572
209.222.82.137    1546
209.222.82.167    1491
209.222.82.164    1488
209.222.82.146    1481
Name: remote_ip, dtype: int64

In [40]:
# We can write a function to let us do this more easily for other fields.
def is_in_top_x(series, x):
    # Returns a series: True if the item is in the most common x values, otherwise False.
    topx_values = series.value_counts().head(x).index.tolist()
    return series.isin(topx_values)

In [41]:
# Set a column in the original DF using this function.
cdf['in_top10_ips'] = is_in_top_x(cdf['remote_ip'], 10)

In [42]:
# We can see how many are/are not within the top 10:
cdf['in_top10_ips'].value_counts()

False    174889
True      25346
Name: in_top10_ips, dtype: int64

In [43]:
# Let's look at the top 10 user-agents too.
cdf['request_header_user_agent'].value_counts().head(10)

Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; Trident/4.0)                                                                                                                                   34929
Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729)                                                         21214
Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.4 Mobile/15E148 Safari/604.1                                                         16951
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36                                                                                5679
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36                                                                                 5573


In [369]:
cdf['in_top10_UAs'] = is_in_top_x(cdf['request_header_user_agent'], 10)

In [370]:
cdf['in_top10_UAs'].value_counts()

False    870
True     630
Name: in_top10_UAs, dtype: int64

In [ ]:
cdf['farm_mid'] = cdf[]

In [44]:
min_timestamp_per_message = cdf.groupby('mid')['request_time'].min()
min_timestamp_per_message.describe()

count                          6609
unique                         6548
top       2019-12-30 15:30:12+00:00
freq                              4
first     2019-12-29 03:13:31+00:00
last      2020-01-26 03:34:31+00:00
Name: request_time, dtype: object

In [46]:
counts_per_message = cdf.mid.value_counts()

In [47]:
counts_per_message.describe()

count     6609.000000
mean        30.297322
std        248.263886
min          1.000000
25%          1.000000
50%          2.000000
75%          5.000000
max      12690.000000
Name: mid, dtype: float64

In [64]:
msgs_over_50 = counts_per_message.query()

AttributeError: 'NoneType' object has no attribute 'query'

In [23]:
counts_per_message = df_counts(cdf, 'mid', head=None)

NameError: name 'df_counts' is not defined

In [24]:
counts_per_message.query('mid_count > 25').shape

NameError: name 'counts_per_message' is not defined